In [1]:
import pandas as pd

In [5]:
df = pd.read_csv(r"C:\Users\82103\Downloads\1cha_선현_general_merge_data_240636_64.csv")

In [6]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from sklearn.preprocessing import MinMaxScaler

# 0. 데이터 정렬 (중요: 회원별, 월별 순서가 맞아야 rolling이 정확합니다)
df = df.sort_values(by=['발급회원번호', '기준년월'])

# -------------------------------------------------------
# 1. 이동 기울기(Rolling Slope) 계산 함수
# -------------------------------------------------------
def calc_slope(y):
    # 데이터가 2개 미만이면 기울기를 구할 수 없음
    if len(y) < 2 or np.sum(y) == 0: 
        return 0.0
    x = np.arange(len(y))
    slope, _, _, _, _ = linregress(x, y)
    return slope

# 최근 3개월(window=3)의 데이터를 사용하여 기울기 산출
# min_periods=2: 데이터가 2개만 쌓여도 기울기 계산 시작
group = df.groupby('발급회원번호')

df['Slope_Spend'] = group['이용금액_신용_B0M'].rolling(window=3, min_periods=2).apply(calc_slope).reset_index(level=0, drop=True)
df['Slope_Balance'] = group['잔액_B0M'].rolling(window=3, min_periods=2).apply(calc_slope).reset_index(level=0, drop=True)
df['Slope_Count'] = group['이용건수_신용_B0M'].rolling(window=3, min_periods=2).apply(calc_slope).reset_index(level=0, drop=True)

# 결측치(첫 달 등)는 0으로 채움
df[['Slope_Spend', 'Slope_Balance', 'Slope_Count']] = df[['Slope_Spend', 'Slope_Balance', 'Slope_Count']].fillna(0)

# -------------------------------------------------------
# 2. 기울기를 '이탈 점수'로 변환 (모든 행에 대해 계산)
# -------------------------------------------------------
scaler = MinMaxScaler()

def convert_slopes_to_scores(dataframe):
    # 음수(감소)인 경우만 추출하여 양수로 변환 (가장 많이 줄어든 게 가장 큰 값)
    # 7월부터 12월까지 모든 행의 기울기를 한꺼번에 스케일링하여 상대적 위치 파악
    temp_scores = {}
    for col in ['Slope_Spend', 'Slope_Balance', 'Slope_Count']:
        neg_val = dataframe[col].apply(lambda x: -x if x < 0 else 0)
        temp_scores[col] = scaler.fit_transform(neg_val.values.reshape(-1, 1)).flatten()
    return temp_scores

# 정규화된 점수 계산 (0.0 ~ 1.0)
norm_scores = convert_slopes_to_scores(df)

# -------------------------------------------------------
# 3. 가중치 적용 및 최종 점수(Churn_Score) 산출
# -------------------------------------------------------
W_SPEND, W_BALANCE, W_COUNT, W_RISK = 40, 30, 20, 10

# 리스크 점수 (연체나 거절이 있으면 1점, 없으면 0점 -> 나중에 가중치 10 곱함)
df['Risk_Flag'] = np.where(
    (df.get('연체잔액_B0M', 0) > 0) | (df.get('승인거절건수_B0M', 0) > 0), 
    1, 0
)

# 최종 스코어 합산
df['Churn_Score'] = (
    (norm_scores['Slope_Spend'] * W_SPEND) + 
    (norm_scores['Slope_Balance'] * W_BALANCE) + 
    (norm_scores['Slope_Count'] * W_COUNT) + 
    (df['Risk_Flag'] * W_RISK)
)

df['Churn_Score'] = df['Churn_Score'].round(1)

# -------------------------------------------------------
# 4. 결과 확인 (최신 월인 12월 데이터만 보기)
# -------------------------------------------------------
target_month = df['기준년월'].max()
df_final = df[df['기준년월'] == target_month].copy()

print(f"\n[{target_month} 기준 이탈 위험 점수 상위 10명]")
cols_view = ['발급회원번호', '기준년월', 'Slope_Spend', 'Churn_Score']
print(df_final[cols_view].sort_values('Churn_Score', ascending=False).head(10))


[201812 기준 이탈 위험 점수 상위 10명]
             발급회원번호    기준년월  Slope_Spend  Churn_Score
203121   SYN_195025  201812    -636369.0         20.2
222077  SYN_1611411  201812    -531305.5         19.2
211969   SYN_852535  201812    -637794.0         18.9
219415  SYN_1406611  201812    -547064.0         16.4
231581  SYN_2331501  201812    -472948.5         16.3
207160   SYN_498091  201812    -524391.5         16.0
207143   SYN_497208  201812    -278261.0         15.8
203725   SYN_240953  201812    -573701.0         14.9
202994   SYN_186121  201812    -521971.0         14.9
230790  SYN_2274803  201812    -452361.5         14.6


In [15]:
# 1. 확인하고 싶은 발급회원번호 설정 (실제 데이터에 있는 번호를 넣으세요)
target_id = df['발급회원번호'].iloc[89765]  # 예시로 첫 번째 회원의 ID를 가져옵니다.

# 2. 해당 회원의 데이터만 추출하여 주요 컬럼 확인
# 새로 만든 컬럼들 + 기준이 되는 컬럼들을 함께 봅니다.
view_cols = [
    '기준년월', 
    '이용금액_신용_B0M', 'Slope_Spend',   # 실제 소비량과 그 기울기
    '잔액_B0M', 'Slope_Balance',        # 잔액과 그 기울기
    'Risk_Flag',                       # 리스크 여부
    'Churn_Score'                      # 최종 이탈 위험 점수
]

customer_history = df[df['발급회원번호'] == target_id][view_cols].sort_values('기준년월')

print(f"🔎 [회원번호: {target_id}] 의 6개월간 이탈 징후 변화")
print("-" * 100)
print(customer_history)
print("-" * 100)

# 3. (팁) 점수가 가장 많이 변한 달 찾기
diff = customer_history['Churn_Score'].max() - customer_history['Churn_Score'].min()
print(f"💡 해당 고객의 6개월간 점수 변동폭: {diff:.1f}점")

🔎 [회원번호: SYN_2017592] 의 6개월간 이탈 징후 변화
----------------------------------------------------------------------------------------------------
          기준년월  이용금액_신용_B0M  Slope_Spend   잔액_B0M  Slope_Balance  Risk_Flag  \
26912   201807      1593210          0.0  2228210            0.0          0   
67018   201808      1310025    -283185.0  1477714      -750496.0          0   
107124  201809      1367279    -112965.5  1644019      -292095.5          0   
147230  201810      1128419     -90803.0   966669      -255522.5          0   
187336  201811      1255182     -56048.5  1253947      -195036.0          0   
227442  201812      1284903      78242.0  1461044       247187.5          0   

        Churn_Score  
26912           0.0  
67018          11.0  
107124          4.2  
147230          3.6  
187336          2.9  
227442          0.3  
----------------------------------------------------------------------------------------------------
💡 해당 고객의 6개월간 점수 변동폭: 11.0점


In [8]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from sklearn.preprocessing import MinMaxScaler

# df 경로는 각자 설정
# df = df.sort_values(by=['발급회원번호', '기준년월'])

# -------------------------------------------------------
# 1. 6개월 기울기(Slope) 계산 (df에 바로 생성)
# -------------------------------------------------------
def calc_slope_6m(series):
    if len(series) < 2 or np.sum(series) == 0: return 0
    y = series.values
    x = np.arange(len(y))
    slope, _, _, _, _ = linregress(x, y)
    return slope

# transform으로 전체 데이터에 기울기 적용
df['Slope_Spend'] = df.groupby('발급회원번호')['이용금액_신용_B0M'].transform(calc_slope_6m)
df['Slope_Balance'] = df.groupby('발급회원번호')['잔액_B0M'].transform(calc_slope_6m)
df['Slope_Count'] = df.groupby('발급회원번호')['이용건수_신용_B0M'].transform(calc_slope_6m)

# -------------------------------------------------------
# 2. 현재 시점(12월) 데이터만 추출
# -------------------------------------------------------
target_month = df['기준년월'].max()
df_final = df[df['기준년월'] == target_month].copy()

# -------------------------------------------------------
# 3. [핵심] 기울기를 '이탈 점수'로 변환 (Normalization)
# -------------------------------------------------------
# 전략: 기울기가 음수일수록(가파른 감소) 높은 점수를 줍니다.
# 양수(증가)인 경우는 이탈 위험이 없으므로 0점으로 처리합니다.

scaler = MinMaxScaler()

def convert_slope_to_score(col_name):
    # 1. 감소하는 경우(음수)만 추출, 나머지는 0으로 대치
    # (음수 값에 -1을 곱해 양수로 만듦: -5000 -> 5000)
    # 즉, 값이 클수록 '많이 줄어듦'을 의미하게 됨
    negative_slopes = df_final[col_name].apply(lambda x: -x if x < 0 else 0)
    
    # 2. 0~1 사이 값으로 정규화 (MinMax Scaling)
    # 가장 많이 줄어든 사람이 1.0, 안 줄어든 사람이 0.0
    return scaler.fit_transform(negative_slopes.values.reshape(-1, 1)).flatten()

# 각 지표별 '감소 심각도' (0.0 ~ 1.0)
score_spend_norm = convert_slope_to_score('Slope_Spend')
score_balance_norm = convert_slope_to_score('Slope_Balance')
score_count_norm = convert_slope_to_score('Slope_Count')

# -------------------------------------------------------
# 4. 가중치 적용 및 최종 점수 산출 (100점 만점)
# -------------------------------------------------------
# 가중치 설정 (비즈니스 중요도에 따라 조절 가능)
W_SPEND = 40   # 소비 감소 비중
W_BALANCE = 30 # 잔액 감소(이탈준비) 비중
W_COUNT = 20   # 빈도 감소 비중
W_RISK = 10    # 현재 연체 등 리스크 비중

# (1) 리스크 점수 별도 계산 (0 or 10점)
# 연체나 승인거절이 하나라도 있으면 10점 만점 부여
has_risk = np.where(
    (df_final.get('연체잔액_B0M', 0) > 0) | 
    (df_final.get('승인거절건수_B0M', 0) > 0), 
    1, 0
)

# (2) 최종 스코어 계산
df_final['Churn_Score'] = (
    (score_spend_norm * W_SPEND) + 
    (score_balance_norm * W_BALANCE) + 
    (score_count_norm * W_COUNT) + 
    (has_risk * W_RISK)
)

# 소수점 정리
df_final['Churn_Score'] = df_final['Churn_Score'].round(1)

# 출력
print("\n[이탈 위험 점수(Churn Score) 상위 10명]")
cols_view = ['발급회원번호', 'Slope_Spend', 'Slope_Balance', 'Churn_Score']
print(df_final[cols_view].sort_values('Churn_Score', ascending=False).head(10))


[이탈 위험 점수(Churn Score) 상위 10명]
         발급회원번호    Slope_Spend  Slope_Balance  Churn_Score
10182  SYN_3968 -243906.428571 -117226.657143         63.7
10493  SYN_5296 -206826.314286 -381888.542857         55.4
11244  SYN_9037 -229686.857143 -119831.371429         54.9
10079  SYN_3442 -168908.285714 -186469.485714         51.4
9667   SYN_1527 -186951.657143 -241112.200000         49.6
10057   SYN_334 -153861.142857 -243873.228571         49.0
9803   SYN_2164 -150169.828571 -159142.771429         48.1
10522  SYN_5434 -161439.171429 -192306.314286         43.3
10539  SYN_5537 -172069.714286 -152554.742857         42.9
10603  SYN_5905 -170207.685714 -156450.228571         41.7


In [9]:
# -------------------------------------------------------
# [추가] 12월 기준 산출된 Score를 전체 과거 데이터(df)에 매핑하기
# -------------------------------------------------------

# 1. df_final에서 '식별자(발급회원번호)'와 '점수(Churn_Score)'만 따로 뽑아냅니다.
# (이게 일종의 "회원별 이탈 점수 매핑 테이블"이 됩니다.)
score_map = df_final[['발급회원번호', 'Churn_Score']]

# 2. 원본 전체 데이터(df)에 점수를 병합합니다.
# on='발급회원번호': 회원번호가 같은 줄을 찾아서,
# how='left': 원본 df의 데이터는 그대로 유지하면서 점수만 오른쪽에 갖다 붙입니다.
df_merged = pd.merge(df, score_map, on='발급회원번호', how='left')

# -------------------------------------------------------
# 결과 확인
# -------------------------------------------------------
print(f"원본 데이터 행 수: {len(df)}")
print(f"병합된 데이터 행 수: {len(df_merged)}") # 원본과 행 수가 같아야 정상입니다.

# 특정 회원 한 명을 찍어서 6개월치 데이터에 점수가 잘 붙었는지 확인
sample_user = df_merged['발급회원번호'].iloc[0] # 첫 번째 회원 예시
print(f"\n[회원번호: {sample_user} 의 데이터 확인]")
cols_check = ['기준년월', '발급회원번호', '이용금액_신용_B0M', 'Churn_Score']
display(df_merged[df_merged['발급회원번호'] == sample_user][cols_check].sort_values('기준년월'))

원본 데이터 행 수: 11460
병합된 데이터 행 수: 11460

[회원번호: SYN_0 의 데이터 확인]


,기준년월,발급회원번호,이용금액_신용_B0M,Churn_Score
0,201807,SYN_0,1420432,4.2
1910,201808,SYN_0,1321346,4.2
3820,201809,SYN_0,1357957,4.2
5730,201810,SYN_0,1344529,4.2
7640,201811,SYN_0,1335154,4.2
9550,201812,SYN_0,1337016,4.2
